# Random Forest Regression Process and Analysis for Solar Data

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
import sklearn.metrics as metrics
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import tree

## Data Preprocessing

First, we read in the dataset.

In [2]:
df = pd.read_csv("../data/solar.csv")
df.head(5)

Now, we must shuffle the datasets to reduce bias.

In [3]:
df = df.sample(frac=1)
df.head(5)

Looking at each dataset, we can identify which variables we want to use
for our models.

In [4]:
X = df.loc[:, ['lat','long','capacity']]
y = df.loc[:, ['generated_energy','cost']]

Now we split into training and testing sets, reserving about 80% for
training and 20% for testing.

In [5]:
X_train = X[:9500]
X_test = X[9500:]
y_train = y[:9500]
y_test = y[9500:]

Models typically perform better when input values are within a certain
range, like \[-1, 1\] for example. We scale the data points
appropriately.

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train

array([[-0.1222636 , -0.72300137, -0.43943183],
       [-0.13674851, -0.24165866, -0.43943988],
       [ 0.68480694, -0.57957659, -0.43942379],
       ...,
       [-0.13809139,  0.21964016, -0.43942379],
       [-0.19139052, -0.60332559, -0.43140258],
       [-0.26111264, -0.03741568, -0.43943988]])

## Training the Models

Now that the data is pre-processed accordingly, the models can be
trained and fit.

In [7]:
reg = RandomForestRegressor()
reg.fit(X_train, y_train)

RandomForestRegressor() In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.   RandomForestRegressor ? Documentation for RandomForestRegressor i Fitted RandomForestRegressor()

With a trained model, predictions can now be made.

In [8]:
display = y_test.reset_index()
preds = reg.predict(X_test)
print("Predictions")
print("----------------------")
for i in range(3):
    print(f"predicted energy: {preds[i][0]:.2f}\tactual energy: {display.at[i, 'generated_energy']:.2f}\tpredicted cost: {preds[i][1]:.2f}\tactual cost: {display.at[i, 'cost']:.2f}")

Predictions
----------------------
predicted energy: 6132.00   actual energy: 6132.00  predicted cost: 5028240.00  actual cost: 5028240.00
predicted energy: 183.96    actual energy: 183.96   predicted cost: 125092.80   actual cost: 125092.80
predicted energy: 12.26 actual energy: 12.26    predicted cost: 9565.92 actual cost: 9565.92

## Testing and Analyzing the Models

This section contains metrics gathering and other figures that visualize
the models and its results.

### Metrics

#### Scores and Error Values

The score being recored are the R2 score, Root Mean Squared Error
(RMSE), and Mean Absolute Percentage Error (MAPE).

In [9]:
r2 = metrics.r2_score(y_test, preds, multioutput="raw_values")
rmse = metrics.root_mean_squared_error(y_test, preds, multioutput="raw_values")
mape = metrics.mean_absolute_percentage_error(y_test, preds, multioutput="raw_values")

print("Metric\tScore")
print("-----------------------")
print(f"r2\t{r2}\nrmse\t{rmse}\nmape\t{mape}")

Metric  Score
-----------------------
r2  [1.         0.99973199]
rmse    [1.84203151e-12 1.00424139e+07]
mape    [9.85642916e-16 4.44841521e-03]

#### Feature Importances

Feature importances give insights into the features that each decision
tree in the random forest use to split most often. Results are portrayed
in percentages.

In [10]:
features = ['lat','long','capacity',]

importances = reg.feature_importances_
indices = np.argsort(importances)

print("Importances")
print('----------------------')
for i in indices:
    print(f"{features[i]}: {importances[i]*100}")

Importances
----------------------
long: 0.5023649924964896
lat: 0.7420038684393504
capacity: 98.75563113906416

#### K-Fold Cross Validation

This cross validation splits up the dataset into 10 unique folds, which
are then used to test a model. The model is then scored using the same
metrics outlined above: R2, RMSE, MAPE. This ensures the scoring is
rigorous, and the *entire* dataset is used.

In [11]:
kf = KFold(n_splits=10, random_state=0, shuffle=True)
kf_cv_scores = cross_validate(reg, X, y, cv=kf, scoring={"r2":metrics.make_scorer(score_func=metrics.r2_score),
 "rmse":metrics.make_scorer(score_func=metrics.root_mean_squared_error),
 "mape":metrics.make_scorer(score_func=metrics.mean_absolute_percentage_error)})
kf_cv_df = pd.DataFrame.from_dict(kf_cv_scores)
means = kf_cv_df.mean()
print("10-Fold Cross Validation Scores")
print("----------------------------------------------------")
print(f"R2 Average: {means.iloc[2]}")
print(f"RMSE Average: {means.iloc[3]}")
print(f"MAPE Average: {means.iloc[4]}")
kf_cv_df

10-Fold Cross Validation Scores
----------------------------------------------------
R2 Average: 0.9997616578182351
RMSE Average: 6164218.5242848415
MAPE Average: 0.0019807060663228687

### Graphs

A graph of the feature importances. This helps to visualize the
magnitude of importance of each feature, and compare their impact
against one another.

In [12]:
plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel("Relative Importance")
plt.show()

A graph of one of the decision trees in the random forest. This displays
the decision making process the model takes to arive at predictions.

In [13]:
fn = ['lat','long','capacity']
cn = ['generated_energy','cost']
plt.subplots(nrows=1, ncols=1, figsize=(4,4), dpi=800)
tree.plot_tree(reg.estimators_[0],feature_names=fn,class_names=cn,filled=True, max_depth=3)
plt.show()